In [1]:
# Agrege acá el código para importar las librerias

# La libreria para "encontrar el sevicio" de Spark
import findspark
findspark.init()

# Librerias para "gestionar el servicio" de Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession

# Creamos una aplicación Spark en el Servicio
# Tenga cuidado con las tildes o caracteres especiales en el nombre de la app
AppSpark = SparkConf().setAppName("Evaluacion iSofware")

# definimos un espacio o contexto para la App
ContextoSpark=SparkContext(conf=AppSpark)

# inicio una sesión en el espacio de la App
SesionSpark = SparkSession(ContextoSpark)

# inicio del espacio o contexto SQL
ContextoSql = SQLContext(sparkContext=ContextoSpark, sparkSession=SesionSpark)

In [2]:
#Copiamos de la carpeta local al Sistema de Archivos
!hdfs dfs -copyFromLocal Asistencia.csv /tmp/

In [6]:
#Copiamos de la carpeta local al Sistema de Archivos
!hdfs dfs -copyFromLocal Evaluacion.csv /tmp/

In [7]:
!hdfs dfs -ls /tmp/

Found 3 items
-rw-r--r--   1 root supergroup      93121 2021-05-13 01:07 /tmp/Asistencia.csv
-rw-r--r--   1 root supergroup     111944 2021-05-13 01:08 /tmp/Evaluacion.csv
drwxr-xr-x   - root supergroup          0 2021-05-13 00:58 /tmp/data


In [8]:
# Consultamos las tablas en SQL Spark
ContextoSql.sql("SHOW TABLES").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [9]:
#2.creamos un dataframe con los datos que descargamos
Asistencia = ContextoSql.read.load("/tmp/Asistencia.csv",
                          format="csv",
                          sep=",",
                          inferSchema="true",
                          header="true")

In [10]:
datax = Asistencia.withColumnRenamed('Nombre de usuario','correo')

In [11]:
datay = datax.withColumnRenamed('Equipo al que perteneces:','EquipoPertenece')


In [12]:
dataz = datay.withColumnRenamed('Equipo que va a exponer:','EquipoExpone')

In [13]:
dataz1 = dataz.withColumnRenamed('Marca temporal','marcatemporal')

In [14]:
dataz1.createOrReplaceTempView('VAsistencia')

In [15]:
ContextoSql.sql("""
select * from VAsistencia
""").show(5)

+--------------------+--------------------+--------------------+--------------------+
|       marcatemporal|              correo|     EquipoPertenece|        EquipoExpone|
+--------------------+--------------------+--------------------+--------------------+
|2021/04/08 8:07:5...|jppinedal@unal.ed...|Pertenezco a: Ser...|Expone: Servicios...|
|2021/04/08 8:09:3...|dgarciabl@unal.ed...|Pertenezco a: Lo ...|Expone: Lo tengo ...|
|2021/04/08 8:09:5...|auarbelaeza@unal....|Pertenezco a: Ges...|Expone: Te lo cam...|
|2021/04/08 8:09:5...|jmunozhe@unal.edu.co|Pertenezco a: Lo ...|Expone: Te lo cam...|
|2021/04/08 8:10:0...|dgerenal@unal.edu.co|Pertenezco a: Te ...|Expone: Te lo cam...|
+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [16]:
ContextoSql.sql("""
select correo,count(*) as Asistencias from VAsistencia group by correo order by 2
""").show()

+--------------------+-----------+
|              correo|Asistencias|
+--------------------+-----------+
|jdroldano@unal.ed...|          1|
|   lmazo@unal.edu.co|          5|
|cquinchiar@unal.e...|          6|
|lbarcelo@unal.edu.co|          7|
|emlopezr@unal.edu.co|          7|
|dosoriom@unal.edu.co|          7|
|jppinedal@unal.ed...|          7|
|mhernandezbe@unal...|          7|
| abanolc@unal.edu.co|          7|
| lrdiaza@unal.edu.co|          7|
|jhcordoba@unal.ed...|          7|
|eporrasm@unal.edu.co|          7|
|hriveraa@unal.edu.co|          7|
|  dbrito@unal.edu.co|          7|
|fguerrerot@unal.e...|          8|
|jgutierrezlo@unal...|          8|
|nvalenciat@unal.e...|          8|
| jruedat@unal.edu.co|          8|
|cjfunezg@unal.edu.co|          8|
|juriveras@unal.ed...|          8|
+--------------------+-----------+
only showing top 20 rows



In [17]:
#2.creamos un dataframe con los datos que descargamos
Evaluacion = ContextoSql.read.load("/tmp/Evaluacion.csv",
                          format="csv",
                          sep=",",
                          inferSchema="true",
                          header="true")

In [25]:
Evaluacion.createOrReplaceTempView('evaluacionx')

In [27]:
Evaluacion.printSchema()

root
 |-- Marca temporal: string (nullable = true)
 |-- Nombre de usuario: string (nullable = true)
 |-- Equipo que vas a evaluar:: string (nullable = true)
 |-- Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?: integer (nullable = true)
 |-- Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?: integer (nullable = true)
 |-- Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?: integer (nullable = true)
 |-- Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?: integer (nullable = true)
 |-- Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?: integer (nullable = true)
 |-- Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.: integer (nullable = true)
 |-- Tracción: El equipo responde adecuadamente si cuenta con clientes que demues

In [23]:
Evaluacion.withColumnRenamed("Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?","introduccion")

DataFrame[Marca temporal: string, Nombre de usuario: string, Equipo que vas a evaluar:: string, introduccion: int, Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?: int, Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?: int, Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?: int, Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?: int, Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.: int, Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.: int, Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.: int, Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?: int, Modelo de negocio: El equipo responde a

In [28]:
df = Evaluacion.withColumnRenamed("Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?","introduccion")
df.createOrReplaceTempView('evaluaciony')

In [29]:
df.printSchema()

root
 |-- Marca temporal: string (nullable = true)
 |-- Nombre de usuario: string (nullable = true)
 |-- Equipo que vas a evaluar:: string (nullable = true)
 |-- introduccion: integer (nullable = true)
 |-- Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?: integer (nullable = true)
 |-- Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?: integer (nullable = true)
 |-- Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?: integer (nullable = true)
 |-- Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?: integer (nullable = true)
 |-- Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.: integer (nullable = true)
 |-- Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.: integer (nullable = true)
 |-- Mercado: El equipo re

In [18]:
Evaluacion.select(['Marca Temporal']).show(10)

+--------------------+
|      Marca Temporal|
+--------------------+
|2021/04/08 8:26:3...|
|2021/04/08 8:26:4...|
|2021/04/08 8:27:2...|
|2021/04/08 8:27:3...|
|2021/04/08 8:27:4...|
|2021/04/08 8:27:5...|
|2021/04/08 8:27:5...|
|2021/04/08 8:27:5...|
|2021/04/08 8:28:1...|
|2021/04/08 8:28:1...|
+--------------------+
only showing top 10 rows



In [33]:
ContextoSql.sql("""
select introduccion from evaluaciony

WHERE introduccion is null
""").show(100)

+------------+
|introduccion|
+------------+
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
+------------+



In [61]:
datax = Evaluacion.withColumnRenamed('Nombre de usuario','correo')

In [62]:
datax.select(['correo']).show(10)

+--------------------+
|              correo|
+--------------------+
|jmunozbo@unal.edu.co|
|daestradam@unal.e...|
|seagudeloo@unal.e...|
|jubuitrago@unal.e...|
|acastrillonv@unal...|
|dgiraldolo@unal.e...|
|jmonsalveb@unal.e...|
|dosoriom@unal.edu.co|
|jlopezpe@unal.edu.co|
|angutierrezb@unal...|
+--------------------+
only showing top 10 rows

